# Read Prisma hyperspectral data from AWS

In [ ]:
import getpass
import hvplot.xarray 
import rioxarray
import fsspec
import rasterio

In [ ]:
s3path = 's3://esip-qhub/usgs/rsignell/testing/PRS_L2D_STD_20200617185821_20200617185825_0001.he5'

In [ ]:
#secret_key = getpass.getpass()  # prompts for the connection string

In [ ]:
from rasterio.session import AWSSession
env = rasterio.Env(AWSSession(region_name='us-west-2', 
                              requester_pays=True,
                             ))

In [ ]:
fs = fsspec.filesystem('s3', anon=False, profile='esip-qhub')

In [ ]:
flist = fs.ls('esip-qhub-push/desis/')
flist

In [ ]:
flist = fs.ls('esip-qhub/usgs/rsignell/testing/')
flist

In [ ]:
fs.get_file(flist[2],'foo.txt')

In [ ]:
!

In [ ]:
fs.get_file(flist[0],'foo.txt')

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%%time
with env:
    with rasterio.open(s3path) as src:
        print(src.profile)
#        ds_list = rioxarray.open_rasterio(src, chunks={'band': -1, 'x': src.width/2, 'y': src.height/2})
        ds_list = rioxarray.open_rasterio(src)

In [ ]:
for (i, ds) in enumerate(ds_list):
    print(i, ds.data_vars)

In [ ]:
ds_list[5].data_vars

In [ ]:
ds = ds_list[5]

In [ ]:
lon = ds_list[1]['HDFEOS_SWATHS_PRS_L2D_HCO_Geolocation_Fields_Longitude'][0,:,:].load()
lat = ds_list[1]['HDFEOS_SWATHS_PRS_L2D_HCO_Geolocation_Fields_Latitude'][0,:,:].load()

In [ ]:
ds = ds.rename({'band':'y', 'y':'band'})

In [ ]:
ds

In [ ]:
ds = ds.assign_coords(lon=(('y', 'x'), lon.values), lat=(('y', 'x'), lat.values))

In [ ]:
ds.data_vars

In [ ]:
%%time
da = ds.HDFEOS_SWATHS_PRS_L2D_HCO_Data_Fields_VNIR_Cube.isel(band=50).load()

In [ ]:
da.hvplot.quadmesh(x='lon', y='lat', rasterize=True, geo=True, frame_width=500,
            cmap='turbo', tiles='OSM', alpha=0.7).redim.nodata(HDFEOS_SWATHS_PRS_L2D_HCO_Data_Fields_VNIR_Cube=0)